## [Meta Coding Puzzles](https://www.metacareers.com/profile/coding_puzzles/)

Below are my solutions for the "Level 2" puzzles.

### Director of Photography (Chapter 2)
Time limit: 5s

*Note: Chapter 1 is an easier version of this puzzle. The only difference is a smaller constraint on $N$.*

A photography set consists of $N$ cells in a row, numbered from $1$ to $N$ in order, and can be represented by a string $C$ of length $N$. Each cell $i$ is one of the following types (indicated by $C_i$, the $i$th character of $C$):

* If $C_i$ = “P”, it is allowed to contain a photographer
* If $C_i$ = “A”, it is allowed to contain an actor
* If $C_i$ = “B”, it is allowed to contain a backdrop
* If $C_i$ = “.”, it must be left empty

A photograph consists of a photographer, an actor, and a backdrop, such that each of them is placed in a valid cell, and such that the actor is between the photographer and the backdrop. Such a photograph is considered artistic if the distance between the photographer and the actor is between $X$ and $Y$ cells (inclusive), and the distance between the actor and the backdrop is also between $X$ and $Y$ cells (inclusive). The distance between cells $i$ and $j$ is $∣i−j∣$ (the absolute value of the difference between their indices).

Determine the number of different artistic photographs which could potentially be taken at the set. Two photographs are considered different if they involve a different photographer cell, actor cell, and/or backdrop cell.

**Constraints**

$1 \leq N \leq 300{,}000 \\
1 \leq X \leq Y \leq N$

**Sample test cases**
1. $N = 5 \\
C = APABA \\
X = 1 \\
Y = 2$

&emsp;&emsp;Expected return value = 1

2. $N = 5 \\
C = APABA \\
X = 2 \\
Y = 3$

&emsp;&emsp;Expected return value = 0

3. $N = 8 \\
C = \, .PBAAP.B \\
X = 1 \\
Y = 3$

&emsp;&emsp;Expected return value = 3

#### Solution

Now, the upper bound on $N$ is $300{,}000$ rather than $200$. My last solution looped through $N^3$ combinations of $P,A,B$ locations, so with a large $N$ this may be too slow given the time limit. Rather than try every combination, I should be able to speed things up by looking only at values I know will be acceptable positions of each $P, A, B$. Rather than checking all $N$ entries for each, I can use `numpy.where()` to extract indices that are potential placements of each element. Since each element $C_i$ can only be one value, each index can only appear for at most one of $P, A, B$. 

I can also reduce the loops by using the artistic photograph conditions. For $P$ at $C_P$, $A$ at $C_A$ has to be within $[C_P - Y, C_P - X]$ or $[C_P + X, C_P + Y]$, i.e. $|C_P - C_A| \geq X$ and $|C_P - C_A| \leq Y$. Similarly, for $B$ at $C_B$, $|C_B - C_A| \geq X$ and $|C_B - C_A| \leq Y$. So, for a given $C_A$, I only have to consider a subset of possible $C_P, C_B$. Also, if $C_P > C_A$, then $C_B < C_A$ is required and vice versa.

This will greatly reduce the number of times through all the loops required.

In [7]:
import numpy as np

In [45]:
# helper function for checking whether a given orientation is a(n artistic) photograph
# to do this, I just need to index of each element and the X and Y values
# first I'll check for valid photograph, returning false if it is not valid
# then I'll check for artistic photograph, returning false if it is not artistic
# finally, if it passes all requirements, return true
def checkArtisticPhotograph(P: int, A: int, B: int, X: int, Y: int) -> bool:
    # check orientation, A needs to be in between P and B
    if P < B and (A > B or A < P):
        return False
    elif B < P and (A < B or A > P):
        return False
    else:
        # orientation valid, check for distances
        if abs(A - P) < X or abs(A - P) > Y:
            return False
        elif abs(A - B) < X or abs(A - B) > Y:
            return False
    # passed all checks
    return True

def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    tStart = time.time()
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed P, A, B
    pVals = np.where(pBool)[0]
    aVals = np.where(aBool)[0]
    bVals = np.where(bBool)[0]
    print(pVals, aVals, bVals)
    
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A, P, B separately
    # limit search to artistic locations of P, B given A
    # skip combinations that don't meet distance and orientation requirements
    # increment counter if all satisfied
    for a in aVals:
        print('Looping for A position '+ str(a) + '. Total time elapsed = ' + str(time.time() - tStart))
        for p in pVals:
            if abs(p - a) < X or abs(p - a) > Y:
                continue
            for b in bVals:
                if abs(b - a) < X or abs(b - a) > Y:
                    continue
                # skip if P, B on same side of A
                if (p > a and b > a) or (p < a and b < a):
                    continue
                nArtisticPhotos += 1
    
    return nArtisticPhotos

In [46]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

[1] [0 2 4] [3]
Looping for A position 0. Total time elapsed = 0.0565030574798584
Looping for A position 2. Total time elapsed = 0.05661296844482422
Looping for A position 4. Total time elapsed = 0.05664801597595215
1
[1] [0 2 4] [3]
Looping for A position 0. Total time elapsed = 0.000286102294921875
Looping for A position 2. Total time elapsed = 0.0003161430358886719
Looping for A position 4. Total time elapsed = 0.0003409385681152344
0
[1 5] [3 4] [2 7]
Looping for A position 3. Total time elapsed = 0.002513885498046875
Looping for A position 4. Total time elapsed = 0.0025548934936523438
3


The simple test cases are solved, but I need to check complex cases where $N$ is at its upper bound. I'll check how long it takes to handle such a case.

In [27]:
import time

In [28]:
import random

In [47]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

[     9     11     19 ... 299986 299990 299991] [     0      2      7 ... 299987 299988 299998] [     1      4      5 ... 299995 299997 299999]
Looping for A position 0. Total time elapsed = 0.10676026344299316
Looping for A position 2. Total time elapsed = 5.9137022495269775
Looping for A position 7. Total time elapsed = 11.443279266357422
Looping for A position 12. Total time elapsed = 16.903674125671387
Looping for A position 16. Total time elapsed = 23.328638315200806
Looping for A position 18. Total time elapsed = 30.022003173828125


KeyboardInterrupt: 

There's still an issue. Even trying to limit the operations, the issue is still with actually looping through everything. It would be nicer possibly if I could change the `for` loop with the given conditions that are checked, rather than checking all ~100,000 values against the conditions.

Some more simplifications I can make:
* $C_A$ has to be at least $X$ cells from the endpoints

In [63]:
def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    tStart = time.time()
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed P, A, B
    pVals = np.where(pBool)[0]
    aVals = np.where(aBool)[0]
    bVals = np.where(bBool)[0]
    # A needs at least X distance to left/right
    aVals = aVals[np.where((aVals >= X) & (aVals <= N-X))[0]]
    print(pVals, aVals, bVals)
    
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A
    # loop through acceptable P, B positions given A
    # and check that they are in the allowed positions for P, B
    # increment counter if all satisfied
    for a in aVals:
        print('Looping for A position '+ str(a) + '. Total time elapsed = ' + str(time.time() - tStart))
        # P to left of A
        for i in range(a-Y, a-X+1):
            if i in pVals:
                # B must be to right of A
                for j in range(a+X, a+Y+1):
                    if j in bVals:
                        nArtisticPhotos += 1
        # P to right of A
        for i in range(a+X, a+Y+1):
            if i in pVals:
                # B must be to left of A
                for j in range(a-Y, a-X+1):
                    if j in bVals:
                        nArtisticPhotos += 1
    
    return nArtisticPhotos

In [61]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

[1] [2 4] [3]
Looping for A position 2. Total time elapsed = 0.001821279525756836
Looping for A position 4. Total time elapsed = 0.006526947021484375
1
[1] [2] [3]
Looping for A position 2. Total time elapsed = 0.004834175109863281
0
[1 5] [3 4] [2 7]
Looping for A position 3. Total time elapsed = 0.002190828323364258
Looping for A position 4. Total time elapsed = 0.0023157596588134766
3


In [62]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

[     0      2      5 ... 299988 299990 299993] [   505    509    511 ... 299492 299493 299500] [     4      9     10 ... 299987 299989 299999]
Looping for A position 505. Total time elapsed = 0.10554075241088867
Looping for A position 509. Total time elapsed = 2.606355905532837
Looping for A position 511. Total time elapsed = 4.94681978225708
Looping for A position 518. Total time elapsed = 7.319139719009399
Looping for A position 519. Total time elapsed = 9.867064714431763
Looping for A position 520. Total time elapsed = 12.272127866744995
Looping for A position 524. Total time elapsed = 14.707591772079468
Looping for A position 529. Total time elapsed = 17.07474994659424
Looping for A position 532. Total time elapsed = 19.482998847961426
Looping for A position 540. Total time elapsed = 21.99131488800049


KeyboardInterrupt: 

Improving (cut down positions to check for $A$, time to check went from ~6-7s to 2-3s), but still too slow.

One improvement, store $P$, $B$ as sets for faster checks/lookups.

But, I don't actually have to do a full loop over positions. Checking one by one is unnecessary if I know that the placements work. $P$ and $B$ are independent of each other. As long as $P$, $B$ individually meet the distance requirements with $A$, it doesn't matter how they relate. Therefore, I don't need to loop over every $B$ for every $P$. I can just count the number of good placements for $B$ and the number of good placements for $P$ for a given $A$ and multiply. This should cut down time going from a nested loop ($N*N$), to looping each separately ($2*N$).

*To try: store cumulative sums of p/b locations so that for a given a position, i can easily calculate the number of p/b in the artistic range without any looping*

In [97]:
from collections import OrderedDict

def getArtisticPhotographCount(N: int, C: str, X: int, Y: int) -> int:
    tStart = time.time()
    # list comprehension gets bool array of each allowed index for P, A, B
    pBool = [True if x=='P' else False for x in C]
    aBool = [True if x=='A' else False for x in C]
    bBool = [True if x=='B' else False for x in C]
    # numpy.where() returns indices for allowed P, A, B
    #pVals = np.where(pBool)[0]
    aVals = np.where(aBool)[0]
    #bVals = np.where(bBool)[0]
    # A needs at least X distance to left/right
    # indices go 0 to N-1, so A should go from 0+X to N-1-X
    aVals = aVals[np.where((aVals >= X) & (aVals <= N-1-X))[0]]
    
    pSums = np.cumsum(pBool)
    bSums = np.cumsum(pBool)
    # store p,b as set for faster lookup
    #pSet = set(pVals)
    #bSet = set(bVals)
    # counter for artistic photographs
    nArtisticPhotos = 0
    
    # now loop through all positions for A
    # loop through acceptable P, B positions given A
    # and check that they are in the allowed positions for P, B
    # increment counter if all satisfied
    for a in aVals:
        #print('Looping for A position '+ str(a) + '. Total time elapsed = ' + str(time.time() - tStart))
        # P to left/right of A
        leftP = pSums[a-X] - pSums[max(0,a-Y)]
        rightP = pSums[min(N-1,a+Y)] - pSums[a+X]
        # B to left/right of A
        leftB = bSums[a-X] - bSums[max(0,a-Y)]
        rightB = bSums[min(N-1,a+Y)] - bSums[a+X]
        # combine left (right) P with right (left) B
        nArtisticPhotos += leftP*rightB
        nArtisticPhotos += rightP*leftB
    
    return nArtisticPhotos

In [90]:
print(getArtisticPhotographCount(5, 'APABA', 1, 2))
print(getArtisticPhotographCount(5, 'APABA', 2, 3))
print(getArtisticPhotographCount(8, '.PBAAP.B', 1, 3))

0
0
2


In [78]:
vals = ['P','A','B','.']
N = 300000
C = random.choices(vals, k=N)
tStart = time.time()
print(getArtisticPhotographCount(N, C, 500, 1000))
tTot = time.time() - tStart
print(tTot)

KeyboardInterrupt: 

In [79]:
C = random.choices(vals, k=100)
pBool = [True if x=='P' else False for x in C]
aBool = [True if x=='A' else False for x in C]
bBool = [True if x=='B' else False for x in C]
# numpy.where() returns indices for allowed P, A, B
pVals = np.where(pBool)[0]
aVals = np.where(aBool)[0]
bVals = np.where(bBool)[0]

In [93]:
pVals[len(pVals)-1] - pVals[67]

IndexError: index 67 is out of bounds for axis 0 with size 27

In [81]:
pBool

[False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False]

In [94]:
pSums = np.cumsum(pBool)

In [95]:
pSums[len(pSums)-1] - pSums[67]

7

In [96]:
np.sum(pBool[67:-1])

7